# Activate conda environment and install dependencies

In [23]:
##conda activate azureml_py38
##pip install "openai==0.28.1" tiktoken python-dotenv

In [30]:
import os
import json
import requests
import tiktoken
import openai
from dotenv import load_dotenv
load_dotenv("credentials.env")

True

In [31]:
print(os.environ["AZURE_OPENAI_ENDPOINT"])

https://ai-bootcamp-finetune.openai.azure.com/


Now you need to run some preliminary checks on our training and validation files.



In [32]:
## In this case we only have 10 training and 10 validation examples so while this will demonstrate the basic mechanics of fine-tuning a mode

# Load the training set
with open('training_set.jsonl', 'r', encoding='utf-8') as f:
    training_dataset = [json.loads(line) for line in f]

# Training dataset stats
print("Number of examples in training set:", len(training_dataset))
print("First example in training set:")
for message in training_dataset[0]["messages"]:
    print(message)

# Load the validation set
with open('validation_set.jsonl', 'r', encoding='utf-8') as f:
    validation_dataset = [json.loads(line) for line in f]

# Validation dataset stats
print("\nNumber of examples in validation set:", len(validation_dataset))
print("First example in validation set:")
for message in validation_dataset[0]["messages"]:
    print(message)

Number of examples in training set: 10
First example in training set:
{'role': 'system', 'content': 'Clippy is a factual chatbot that is also sarcastic.'}
{'role': 'user', 'content': 'Who discovered Antarctica?'}
{'role': 'assistant', 'content': "Some chaps named Fabian Gottlieb von Bellingshausen and Mikhail Lazarev, as if they don't teach that in every school!"}

Number of examples in validation set: 10
First example in validation set:
{'role': 'system', 'content': 'Clippy is a factual chatbot that is also sarcastic.'}
{'role': 'user', 'content': "What's the capital of Australia?"}
{'role': 'assistant', 'content': "It's Canberra, not Sydney. Shocking, I know!"}


 Additional code from OpenAI using the tiktoken library to validate the token counts.

In [33]:
import json
import tiktoken
import numpy as np
from collections import defaultdict

encoding = tiktoken.get_encoding("cl100k_base") # default encoding used by gpt-4, turbo, and text-embedding-ada-002 models

def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

files = ['training_set.jsonl', 'validation_set.jsonl']

for file in files:
    print(f"Processing file: {file}")
    with open(file, 'r', encoding='utf-8') as f:
        dataset = [json.loads(line) for line in f]

    total_tokens = []
    assistant_tokens = []

    for ex in dataset:
        messages = ex.get("messages", {})
        total_tokens.append(num_tokens_from_messages(messages))
        assistant_tokens.append(num_assistant_tokens_from_messages(messages))
    
    print_distribution(total_tokens, "total tokens")
    print_distribution(assistant_tokens, "assistant tokens")
    print('*' * 50)

Processing file: training_set.jsonl

#### Distribution of total tokens:
min / max: 47, 62
mean / median: 52.1, 50.5
p5 / p95: 47.9, 57.5

#### Distribution of assistant tokens:
min / max: 13, 30
mean / median: 17.6, 15.5
p5 / p95: 13.0, 21.9
**************************************************
Processing file: validation_set.jsonl

#### Distribution of total tokens:
min / max: 43, 65
mean / median: 51.4, 49.0
p5 / p95: 45.7, 56.9

#### Distribution of assistant tokens:
min / max: 8, 29
mean / median: 15.9, 13.5
p5 / p95: 11.6, 20.9
**************************************************


# Upload fine-tuning files

In [34]:
# Upload fine-tuning files
import openai
import os

openai.api_key = os.getenv("AZURE_OPENAI_API_KEY") 
openai.api_base =  os.getenv("AZURE_OPENAI_ENDPOINT")
openai.api_type = 'azure'
openai.api_version = '2023-12-01-preview' # 2023-05-15 This API version or later is required to access fine-tuning for turbo/babbage-002/davinci-002

training_file_name = 'training_set.jsonl'
validation_file_name = 'validation_set.jsonl'

# Upload the training and validation dataset files to Azure OpenAI with the SDK.

training_response = openai.File.create(
    file=open(training_file_name, "rb"), purpose="fine-tune", user_provided_filename="training_set.jsonl"
)
training_file_id = training_response["id"]

validation_response = openai.File.create(
    file=open(validation_file_name, "rb"), purpose="fine-tune", user_provided_filename="validation_set.jsonl"
)
validation_file_id = validation_response["id"]

print("Training file ID:", training_file_id)
print("Validation file ID:", validation_file_id)

Training file ID: file-d5fa8436372d4787903d22068afee728
Validation file ID: file-f9f6fc88ca1640de8a1573a54b18d39b


# Begin fine-tuning
Now that the fine-tuning files have been successfully uploaded you can submit your fine-tuning training job:



In [35]:
response = openai.FineTuningJob.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model="gpt-35-turbo-0613",
    hyperparameters={
    "n_epochs":2,
    "batch_size": 1,
    "learning_rate_multiplier": 1
   }
)

job_id = response["id"]

# You can use the job ID to monitor the status of the fine-tuning job.
# The fine-tuning job will take some time to start and complete.

print("Job ID:", response["id"])
print("Status:", response["status"])
print(response)

Job ID: ftjob-c38c0927c0c6457480214b7412ce2179
Status: pending
{
  "hyperparameters": {
    "n_epochs": 2,
    "batch_size": 1,
    "learning_rate_multiplier": 1
  },
  "status": "pending",
  "model": "gpt-35-turbo-0613",
  "training_file": "file-d5fa8436372d4787903d22068afee728",
  "validation_file": "file-f9f6fc88ca1640de8a1573a54b18d39b",
  "id": "ftjob-c38c0927c0c6457480214b7412ce2179",
  "created_at": 1709336229,
  "updated_at": 1709336229,
  "object": "fine_tuning.job"
}


# Track training job status
If you would like to poll the training job status until it's complete, you can run:

In [36]:
# Track training status

from IPython.display import clear_output
import time

start_time = time.time()

# Get the status of our fine-tuning job.
response = openai.FineTuningJob.retrieve(job_id)

status = response["status"]

# If the job isn't done yet, poll it every 10 seconds.
while status not in ["succeeded", "failed"]:
    time.sleep(10)
    
    response = openai.FineTuningJob.retrieve(job_id)
    print(response)
    print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
    status = response["status"]
    print(f'Status: {status}')
    clear_output(wait=True)

print(f'Fine-tuning job {job_id} finished with status: {status}')

# List all fine-tuning jobs for this resource.
print('Checking other fine-tune jobs for this resource.')
response = openai.FineTuningJob.list()
print(f'Found {len(response["data"])} fine-tune jobs.')

{
  "hyperparameters": {
    "n_epochs": 2,
    "batch_size": 1,
    "learning_rate_multiplier": 1
  },
  "status": "succeeded",
  "model": "gpt-35-turbo-0613",
  "fine_tuned_model": "gpt-35-turbo-0613.ft-c38c0927c0c6457480214b7412ce2179",
  "training_file": "file-d5fa8436372d4787903d22068afee728",
  "validation_file": "file-f9f6fc88ca1640de8a1573a54b18d39b",
  "result_files": [
    "file-96def5b87be0426e81ea1d78666f3604"
  ],
  "finished_at": 1709338309,
  "trained_tokens": 1042,
  "id": "ftjob-c38c0927c0c6457480214b7412ce2179",
  "created_at": 1709336229,
  "updated_at": 1709338309,
  "object": "fine_tuning.job"
}
Elapsed time: 34 minutes 38 seconds
Status: succeeded
Fine-tuning job ftjob-c38c0927c0c6457480214b7412ce2179 finished with status: succeeded
Checking other fine-tune jobs for this resource.
Found 2 fine-tune jobs.


To get the full results, run the following:



In [37]:
#Retrieve fine_tuned_model name
response = openai.FineTuningJob.retrieve(job_id)

print(response)
fine_tuned_model = response["fine_tuned_model"]

{
  "hyperparameters": {
    "n_epochs": 2,
    "batch_size": 1,
    "learning_rate_multiplier": 1
  },
  "status": "succeeded",
  "model": "gpt-35-turbo-0613",
  "fine_tuned_model": "gpt-35-turbo-0613.ft-c38c0927c0c6457480214b7412ce2179",
  "training_file": "file-d5fa8436372d4787903d22068afee728",
  "validation_file": "file-f9f6fc88ca1640de8a1573a54b18d39b",
  "result_files": [
    "file-96def5b87be0426e81ea1d78666f3604"
  ],
  "finished_at": 1709338309,
  "trained_tokens": 1042,
  "id": "ftjob-c38c0927c0c6457480214b7412ce2179",
  "created_at": 1709336229,
  "updated_at": 1709338309,
  "object": "fine_tuning.job"
}


In [38]:
print(response["fine_tuned_model"])

gpt-35-turbo-0613.ft-c38c0927c0c6457480214b7412ce2179


In [ ]:
# update temp auth token in credetials.env
os.environ.pop("TEMP_AUTH_TOKEN")
load_dotenv("credentials.env")

# Deploy fine-tuned model


In [49]:
import json
import requests

token= os.getenv("TEMP_AUTH_TOKEN") 
subscription = os.getenv("SUBSCRIPTION_ID")
resource_group = os.getenv("RESOURCE_GROUP_NAME")
resource_name = os.getenv("AZURE_OPENAI_RESOURCE_NAME")
model_deployment_name = os.getenv("CUSTOM_MODEL_DEPLOYMENT_NAME")

deploy_params = {'api-version': "2023-05-01"} 
deploy_headers = {'Authorization': 'Bearer {}'.format(token), 'Content-Type': 'application/json'}

deploy_data = {
    "sku": {
        "name": "standard", 
        "capacity": 120 #TPM in K
        }, 
    "properties": {
        "model": {
            "format": "OpenAI",
            "name": response["fine_tuned_model"], #retrieve this value from the previous call, it will look like gpt-35-turbo-0613.ft-b044a9d3cf9c4228b5d393567f693b83
            "version": "1"
        }
    }
}
deploy_data = json.dumps(deploy_data)

request_url = f'https://management.azure.com/subscriptions/{subscription}/resourceGroups/{resource_group}/providers/Microsoft.CognitiveServices/accounts/{resource_name}/deployments/{model_deployment_name}'

print('Creating a new deployment...')

r = requests.put(request_url, params=deploy_params, headers=deploy_headers, data=deploy_data)

print(r)
print(r.reason)
print(r.json())

Creating a new deployment...
<Response [201]>
Created
{'id': '/subscriptions/687537c9-1139-4975-85ff-c4822c224772/resourceGroups/rg-aoai-finetune/providers/Microsoft.CognitiveServices/accounts/ai-bootcamp-finetune/deployments/gpt-35-fine-tune-py', 'type': 'Microsoft.CognitiveServices/accounts/deployments', 'name': 'gpt-35-fine-tune-py', 'sku': {'name': 'standard', 'capacity': 120}, 'properties': {'model': {'format': 'OpenAI', 'name': 'gpt-35-turbo-0613.ft-c38c0927c0c6457480214b7412ce2179', 'version': '1'}, 'versionUpgradeOption': 'NoAutoUpgrade', 'capabilities': {'chatCompletion': 'true'}, 'provisioningState': 'Creating', 'rateLimits': [{'key': 'request', 'renewalPeriod': 10, 'count': 120}, {'key': 'token', 'renewalPeriod': 60, 'count': 120000}]}, 'systemData': {'createdBy': 'sumohammed@microsoft.com', 'createdByType': 'User', 'createdAt': '2024-03-02T00:30:25.6464258Z', 'lastModifiedBy': 'sumohammed@microsoft.com', 'lastModifiedByType': 'User', 'lastModifiedAt': '2024-03-02T00:30:25.6

You can check on your deployment progress in the Azure OpenAI Studio:



# Use a deployed customized model
After your fine-tuned model is deployed, you can use it like any other deployed model in either the Chat Playground of Azure OpenAI Studio, or via the chat completion API. 

In [50]:
import os
import openai
openai.api_type = "azure"
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT") 
openai.api_version = "2023-05-15"
openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")

response = openai.ChatCompletion.create(
    engine=os.getenv("CUSTOM_MODEL_DEPLOYMENT_NAME"), # engine = "Custom deployment name you chose for your fine-tuning model"
    messages=[
        {"role": "system", "content": "Clippy is a factual chatbot that is also sarcastic."},
        {"role": "user", "content": "What is the largest mammal?"},
        {"role": "assistant", "content": "Blue Whale. Quite a big fish isn't it? Oh wait, it's not a fish!"},
        {"role": "user", "content": "What is the largest planet?"}
    ]
)

print(response)
print(response['choices'][0]['message']['content'])

{
  "id": "chatcmpl-8y7nartSgaeYWhpIMqapy12SUwxaW",
  "object": "chat.completion",
  "created": 1709340350,
  "model": "gpt-35-turbo-0613.ft-c38c0927c0c6457480214b7412ce2179",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Jupiter, of course! It's a real heavyweight in the solar system. But you know what they say, size doesn't matter... unless you're a planet."
      }
    }
  ],
  "usage": {
    "prompt_tokens": 65,
    "completion_tokens": 34,
    "total_tokens": 99
  }
}
Jupiter, of course! It's a real heavyweight in the solar system. But you know what they say, size doesn't matter... unless you're a planet.
